In [1]:
from __future__ import print_function
import os.path
import dalmatian as dm
import pandas as pd
import sys
from src.CCLE_postp_function import *
sys.path.insert(0, '../JKBio/')
import Datanalytics as da 
import TerraFunction as terra
%load_ext autoreload
%autoreload 2
%load_ext rpy2.ipython
from taigapy import TaigaClient
from IPython.display import Image,display
tc = TaigaClient()
import numpy as np
from Bio import AlignIO

In [2]:
refnamespace="broad-firecloud-ccle"
refworkspace="DepMap_Mutation_Calling_CGA_pipeline"

# Downloading from terra

In [4]:
refwm= dm.WorkspaceManager(refnamespace,refworkspace)

In [5]:
res = refwm.get_sample_sets().loc['DM19Q2_COMPLETE_SET']
res

aggregateMAF                                                                      nan
filtered_CGA_MAF_aggregated                                                       nan
passedCGA_filteredMAF_aggregated    gs://fc-secure-012d088c-f039-4d36-bde5-ee9b1b7...
samples                                                                            []
unfiltered_CGA_MAF_aggregated       gs://fc-secure-012d088c-f039-4d36-bde5-ee9b1b7...
Name: DM19Q2_COMPLETE_SET, dtype: object

In [10]:
filtered = res['passedCGA_filteredMAF_aggregated']
unfiltered = res['unfiltered_CGA_MAF_aggregated']

! gsutil cp $filtered "temp/mutation_filtered_terra_merged.txt"
! gsutil cp $unfiltered "temp/mutation_unfiltered_terra_merged.txt"

Copying gs://fc-secure-012d088c-f039-4d36-bde5-ee9b1b76b912/aa8cbc26-e1f1-4100-b424-e2a461d6f10d/aggregateMAFs_selectFields_workflow/f25a70f4-56e3-4e3c-87f4-75ad36c090f4/call-aggregateMAFs_selectFields/DM19Q2_COMPLETE_SET.mergedMAF.txt...
- [1 files][ 92.3 MiB/ 92.3 MiB]    4.3 MiB/s                                   
Operation completed over 1 objects/92.3 MiB.                                     
Copying gs://fc-secure-012d088c-f039-4d36-bde5-ee9b1b76b912/7bf15e7f-69d6-474c-b335-2c9848cf018c/aggregateMAFs_selectFields_workflow/8961929c-7fb0-4077-82cd-7264015c46a7/call-aggregateMAFs_selectFields/attempt-2/DM19Q2_COMPLETE_SET.mergedMAF.txt...
Resuming download for temp/mutation_unfiltered_terra_merged.txt component 0     
Resuming download for temp/mutation_unfiltered_terra_merged.txt component 1
Resuming download for temp/mutation_unfiltered_terra_merged.txt component 3
Resuming download for temp/mutation_unfiltered_terra_merged.txt component 2
| [1 files][858.2 MiB/858.2 MiB]    3.5 

In [7]:
release = pd.read_csv('temp/mutations.19Q3.tsv', sep='\t')

/anaconda3/envs/py36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (3,19,25,26,27,28,29,30,31) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [8]:
len(release)

1277316

In [9]:
filtered = pd.read_csv('temp/mutation_filtered_terra_merged.txt', sep='\t')
unfiltered = pd.read_csv('temp/mutation_unfiltered_terra_merged.txt', sep='\t')

In [10]:
len(filtered)

411053

In [27]:
unfiltered = unfiltered[set(unfiltered.columns.tolist()) & set(filtered.columns.tolist())]
filtered = filtered[unfiltered.columns]

## How much genome change non overlap do we have to the filtered version?

{{b}}

In [41]:
b = len(set(filtered.Genome_Change.tolist())) - len(set(unfiltered.Genome_Change.tolist()) & set(filtered.Genome_Change.tolist()))
b

0

## how much genome change do we have in duplicate? 
R : {{a*100}}%

In [40]:
a = (len(filtered) - len(set(filtered.Genome_Change.tolist()))) / len(filtered)
a

0.0755741960282494

In [47]:
len(filtered[filtered.Hugo_Symbol=='TP53']) - len(release[release.Hugo_Symbol=='TP53'])

-809

In [48]:
release[release.Hugo_Symbol=='TP53']

,Hugo_Symbol,Entrez_Gene_Id,NCBI_Build,Chromosome,Start_position,End_position,Strand,Variant_Classification,Variant_Type,Reference_Allele,...,isCOSMIChotspot,COSMIChsCnt,ExAC_AF,CGA_WES_AC,SangerWES_AC,SangerRecalibWES_AC,RNAseq_AC,HC_AC,RD_AC,WGS_AC
96,TP53,7157,37,17,7577538,7577538,+,Missense_Mutation,SNP,C,...,False,0.0,0.000058,NaN,20:0,20:0,302:4,70:1,NaN,19:0
581,TP53,7157,37,17,7578239,7578239,+,Nonsense_Mutation,SNP,C,...,True,63.0,NaN,NaN,NaN,NaN,8:1,NaN,NaN,NaN
800,TP53,7157,37,17,7578532,7578532,+,Missense_Mutation,SNP,A,...,True,37.0,NaN,NaN,NaN,23:0,362:0,25:0,NaN,NaN
1091,TP53,7157,37,17,7578532,7578532,+,Missense_Mutation,SNP,A,...,True,37.0,NaN,153:0,NaN,NaN,340:0,22:0,274:0,23:0
1341,TP53,7157,37,17,7577120,7577120,+,Missense_Mutation,SNP,C,...,True,669.0,0.000025,NaN,22:0,23:0,381:4,11:0,235:1,NaN
2577,TP53,7157,37,17,7578239,7578239,+,Nonsense_Mutation,SNP,C,...,True,63.0,NaN,NaN,28:0,28:0,10:2,240:0,NaN,27:0
2898,TP53,7157,37,17,7578500,7578500,+,Nonsense_Mutation,SNP,G,...,True,65.0,0.000008,NaN,NaN,NaN,22:0,NaN,NaN,NaN
3524,TP53,7157,37,17,7578195,7578197,+,In_Frame_Del,DEL,CAC,...,True,62.0,NaN,NaN,33:2,40:2,192:0,83:8,NaN,44:8
3774,TP53,7157,37,17,7577538,7577538,+,Missense_Mutation,SNP,C,...,False,0.0,0.000058,0:0,NaN,NaN,594:4,72:0,NaN,NaN
4217,TP53,7157,37,17,7577098,7577098,+,Missense_Mutation,SNP,T,...,True,43.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7:32


## Looking at data from 19Q1 to see exactly the missign mutations

In [11]:
new = tc.get(name='depmap-mutation-calls-9be3', version=15, file='depmap_19Q3_mutation_calls')
prev = tc.get(name='depmap-mutation-calls-9be3', version=11, file='depmap_19Q1_mutation_calls')

[##################]100% |   4.7 MiB/s | 282.6 MiB / 282.6 MiB | Time:  0:01:00
/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:1: DtypeWarning: Columns (3,19,25,26,27,28,29,30,31) have mixed types. Specify dtype option on import or set low_memory=False.
  """Entry point for launching an IPython kernel.
[##################]100% |   5.8 MiB/s | 298.7 MiB / 298.7 MiB | Time:  0:00:51
/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:2: DtypeWarning: Columns (4,20,23,26,27,28,29,30,31,32,33) have mixed types. Specify dtype option on import or set low_memory=False.
  
